In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
import plotly.express as px
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.distributions.empirical_distribution import ECDF
import datetime
import pandas as pd
import numpy as np
import pickle


df=pd.read_csv('https://github.com/RCML2016/TimeSeriesAnalysis_StoreProductForecast/blob/master/Sales_Product_Price_by_Store.csv?raw=true',sep=',')
#df


In [ ]:
#import io
#df2 = pd.read_csv(io.BytesIO(uploaded['Sales_Product_Price_by_Store.csv']))
# Dataset is now stored in a Pandas Dataframe
#df = pd.read_csv('Sales_Product_Price_by_Store.csv')

In [ ]:
df['Date']=pd.to_datetime(df['Date'])
df['weekly_sales']=df['Price'] * df['Weekly_Units_Sold']
df.head()

In [ ]:
df.set_index('Date', inplace=True)
df['year']=df.index.year
df['month']=df.index.month
df['day']=df.index.day
df['week_of_year']=df.index.weekofyear
df.head()#

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='ticks')
c= '#386B7F'
figure, axes=plt.subplots(nrows=2, ncols=2)
figure.tight_layout(pad=2.0)
plt.subplot(211)
cdf=ECDF(df['Weekly_Units_Sold'])
plt.plot(cdf.x,cdf.y,label="statmodels",color=c);
plt.xlabel('weekly Units Sold'); plt.ylabel('ECDF');

plt.subplot(212)
cdf=ECDF(df['weekly_sales'])
plt.plot(cdf.x,cdf.y,label="statmodels",color=c);
plt.xlabel('Weekly Sales'); plt.ylabel('ECDF');

In [ ]:
df.groupby('Store')['weekly_sales'].describe()

In [ ]:
df.groupby('Store')['Weekly_Units_Sold'].sum()

In [ ]:
g = sns.FacetGrid(df, col="Is_Holiday", height=4, aspect=.8)
g.map(sns.barplot,"Product","Price")

Product 2 is the cheapest product among these three products, so, it sells the most. Product 3 is the most expensive product among these three. Product price did not change during holidays.

Because we have recorded holidays sales, so we will analyze if holiday also contributed to the sales.

In [ ]:
g = sns.FacetGrid(df, row="Is_Holiday",
                  height=1.7, aspect=4,)
g.map(sns.distplot, "Weekly_Units_Sold", hist=False, rug=True)

In [ ]:
sns.factorplot(data=df,
               x= 'Is_Holiday',
               y='Weekly_Units_Sold',
               hue='Store')

sns.factorplot(data= df, 
               x= 'Is_Holiday',
               y= 'Weekly_Units_Sold',
               hue= 'Product')

From the above figures we can see that holidays do not have a positive impact on the business. For most of the stores, weekly unit sales on the holidays is as same as the normal days, while store 10 also face a decrease in sales during the holidays.

# New Section

Weekly units sold for product 1 had a slightly increase during the holidays, while product 2 and product 3 had a decrease during the holidays.

In [ ]:
g = sns.FacetGrid(df, col="Product", row="Is_Holiday", margin_titles=True, height=3)
g.map(plt.scatter, "Price", "Weekly_Units_Sold", color="#338844", edgecolor="white", s=50, lw=1)
g.set(xlim=(0, 30), ylim=(0, 2600));

Every product has more than one price, both in holidays and normal days. One price is regular price, and another is a promotional price. However, the price gap for product 3 is huge, it was slashed to almost 50% off during promotions.

Product 3 made the most sales during normal days.



In [ ]:
g = sns.FacetGrid(df, col="Store", hue="Product", margin_titles=True, col_wrap=3)
g.map(plt.scatter, 'Price', 'Weekly_Units_Sold', alpha=.7)
g.add_legend()

All the stores have the similar price promotion pattern, for some reason, Store 10 sells the most during the promotions. All the products have the regular price and promotion price. Product 3 has the highest discount and sells the most during the promotions.

In [ ]:
#df.groupby(['Product', 'promotion'])['Price', 'Weekly_Units_Sold'].mean()

In [ ]:
corr_all = df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_all, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize = (11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_all, mask = mask,
            square = True, linewidths = .5, ax = ax, cmap = "BuPu")      
plt.show();

We have a strong positive correlation between price and Base price, weekly units sold and weekly sales, base price and product, price and product. We can also observe a positive correlation between month and week of the year.

Observations of our EDA:

The most selling and crowded Store is Store 10, and the least crowded store is Store 5.
In terms of number of units sold, the most selling product is product 2. In terms of sales dollars, Product 3 posts the highest sales during normal days.
Stores do not necessarily run product promotions during holidays. Holidays do not seem to have an impact on Stores’ performance.
Product 1 sells a little more during holidays, however, Product 2 and Product 3 sell less on holidays.
Product 2 seems to be the cheapest product, and Product 3 is the most expensive product.
Most stores have some kind of seasonality and they make the highest sales around July.
Product 1 sells a little more in February than the other months, Product 2 sells the most around April and July, and Product 3 sells the most around July.
In general, product 2 sells the most at Store 10, but in July, Product 3 has the highest sales in this store.
Each product has its regular price and promotional price. There isn’t significant gap between regular price and promotional price on Product 1 and Product 2, however, Product 3’s promotional price can be slashed to 50% of its original price. Although every store makes this kind of price cut for product 3, Store 10 is the one made the highest sales during the price cut.
It is nothing unusual to sell more during promotion than the normal days. Store 10’s made Product 3 the best selling product around July.

Time Series Forecasting and Sales Prediction
Now let’s move to the Time Series Forecasting Part of this article, here we will forecast sales, according to our above observations of exploratory data analysis.

In [ ]:
# store types
sales_1 = df[df.Store == 1]['weekly_sales']
sales_2 = df[df.Store == 2]['weekly_sales']
sales_3 = df[df.Store == 3]['weekly_sales']
sales_4 = df[df.Store == 4]['weekly_sales']
sales_5 = df[df.Store == 5]['weekly_sales']
sales_6 = df[df.Store == 6]['weekly_sales']
sales_7 = df[df.Store == 7]['weekly_sales']
sales_8 = df[df.Store == 8]['weekly_sales']
sales_10 = df[df.Store == 10]['weekly_sales']

f, (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9) = plt.subplots(9, figsize = (20, 15))

# store types
sales_1.plot(color = c, ax = ax1)
sales_2.plot(color = c, ax = ax2)
sales_3.plot(color = c, ax = ax3)
sales_4.plot(color = c, ax = ax4)
sales_5.plot(color = c, ax = ax5)
sales_6.plot(color = c, ax = ax6)
sales_7.plot(color = c, ax = ax7)
sales_8.plot(color = c, ax = ax8)
sales_10.plot(color = c, ax = ax9)

Time Series Forecasting
Time Series of the weekly sales:

In [ ]:
store_10_pro_3 = df[(df.Store == 10) & (df.Product == 3)].loc[:, ['Base Price', 'Price', 'Weekly_Units_Sold', 'weekly_sales']]
store_10_pro_3.reset_index(level=0, inplace=True)
fig = px.line(store_10_pro_3, x='Date', y='weekly_sales')
fig.update_layout(title_text='Time Series of weekly sales')
fig.show()

Product 2’s seasonality at store 10 is obvious. The sales always peak between July and September during school holiday. Below we are implementing prophet model, forecasting the weekly sales for the future 50 weeks.

In [ ]:
store_10_pro_3 = store_10_pro_3[['Date', 'weekly_sales']].rename(columns = {'Date': 'ds',
                                'weekly_sales': 'y'})

In [ ]:
model = Prophet(interval_width = 0.95)
model.fit(store_10_pro_3)

future_dates = model.make_future_dataframe(periods = 50, freq='W')

future_dates.tail(7)

In [ ]:
forecast = model.predict(future_dates)

# preditions for last week
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
model.plot(forecast)

In [ ]:
model.plot_components(forecast)

In [ ]:
metric_df = forecast.set_index('ds')[['yhat']].join(store_10_pro_3.set_index('ds').y).reset_index()
metric_df.dropna(inplace=True)
error = mean_squared_error(metric_df.y, metric_df.yhat)
print('The RMSE is {}'. format(sqrt(error)))

In [ ]:
with open ('StorePrdctTsPredection.pkl','wb') as f:
    pickle.dump(forecast,f)